In [2]:
#load necessary modules
#!pip3 install torch
#!pip3 install transformers
#!pip3 install sentence_transformers

In [4]:
# install gradio for user app
#!pip3 install gradio


     ---------------------------------------- 14.2/14.2 MB 6.3 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 5.8 MB/s eta 0:00:00
     -------------------------------------- 324.5/324.5 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 56.2/56.2 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 11.0/11.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 56.9/56.9 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 7.2/7.2 MB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 71.5/71.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 813.6/813.6 kB 5.7 MB/s eta 0:00:00
  

In [17]:
!pip3 install pyarrow

     ---------------------------------------- 21.5/21.5 MB 5.2 MB/s eta 0:00:00


In [17]:
#!pip3 install unidecode
#!pip3 install skimpy

In [1]:
import torch
import transformers
import sentence_transformers
import numpy as np
import pandas as pd



In [2]:
# load data
soc=pd.read_csv(r"C:\dataset\NFVI_SOC.csv", encoding='cp1252')

In [3]:
#glimpse the data
soc.head()

,Domain,Solution Area,Product,Produt Release,Solution Category,Deployment Model,Requirement,Response,Compliance,Full Bid Name,Bid ID Only,Day marked completed,Customer,Country,Market Area,bid_type,Responded By
0,Cloud,NFVI,Solution,7,NFVI,NFVI,Restrict containers running in privileged mode,Containers can be restricted to be running in ...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
1,Cloud,NFVI,Solution,7,NFVI,NFVI,Do not run processes in containers as root,Presently docker deamon is running as root but...,NC,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
2,Cloud,NFVI,Solution,7,NFVI,NFVI,Restrict the use of hostpath,It is possible for a container to breakout of ...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
3,Cloud,NFVI,Solution,7,NFVI,NFVI,"Where possible, Cryptographically isolate crit...",Where possible Ericsson will try to implement ...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
4,Cloud,NFVI,Solution,7,NFVI,NFVI,Use the PodSpec to set limits to help minimize...,Ericsson uses CPU pinning and resource dedicat...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav


In [4]:
soc.shape

(4155, 17)

In [5]:
#fix the messy column names
from skimpy import clean_columns
import skimpy
soc_clean = clean_columns(soc)

16 column names have been cleaned

In [6]:
#make all cols as str
for col in soc_clean.columns:
    soc_clean[col] = soc_clean[col].astype(str)

In [7]:
#remove the leading spaces
soc_clean = soc_clean.applymap(lambda x: x.lstrip() if isinstance(x, str) else x)


In [8]:
soc_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4155 entries, 0 to 4154
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   domain                4155 non-null   object
 1   solution_area         4155 non-null   object
 2   product               4155 non-null   object
 3   produt_release        4155 non-null   object
 4   solution_category     4155 non-null   object
 5   deployment_model      4155 non-null   object
 6   requirement           4155 non-null   object
 7   response              4155 non-null   object
 8   compliance            4155 non-null   object
 9   full_bid_name         4155 non-null   object
 10  bid_id_only           4155 non-null   object
 11  day_marked_completed  4155 non-null   object
 12  customer              4155 non-null   object
 13  country               4155 non-null   object
 14  market_area           4155 non-null   object
 15  bid_type              4155 non-null   

In [9]:
soc_clean.head()

,domain,solution_area,product,produt_release,solution_category,deployment_model,requirement,response,compliance,full_bid_name,bid_id_only,day_marked_completed,customer,country,market_area,bid_type,responded_by
0,Cloud,NFVI,Solution,7,NFVI,NFVI,Restrict containers running in privileged mode,Containers can be restricted to be running in ...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
1,Cloud,NFVI,Solution,7,NFVI,NFVI,Do not run processes in containers as root,Presently docker deamon is running as root but...,NC,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
2,Cloud,NFVI,Solution,7,NFVI,NFVI,Restrict the use of hostpath,It is possible for a container to breakout of ...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
3,Cloud,NFVI,Solution,7,NFVI,NFVI,"Where possible, Cryptographically isolate crit...",Where possible Ericsson will try to implement ...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
4,Cloud,NFVI,Solution,7,NFVI,NFVI,Use the PodSpec to set limits to help minimize...,Ericsson uses CPU pinning and resource dedicat...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav


In [13]:
#slice the dataset to small using 800 rows
soc_small=soc_clean[:1000]
soc_small.head()


,domain,solution_area,product,produt_release,solution_category,deployment_model,requirement,response,compliance,full_bid_name,bid_id_only,day_marked_completed,customer,country,market_area,bid_type,responded_by
0,Cloud,NFVI,Solution,7,NFVI,NFVI,Restrict containers running in privileged mode,Containers can be restricted to be running in ...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
1,Cloud,NFVI,Solution,7,NFVI,NFVI,Do not run processes in containers as root,Presently docker deamon is running as root but...,NC,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
2,Cloud,NFVI,Solution,7,NFVI,NFVI,Restrict the use of hostpath,It is possible for a container to breakout of ...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
3,Cloud,NFVI,Solution,7,NFVI,NFVI,"Where possible, Cryptographically isolate crit...",Where possible Ericsson will try to implement ...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
4,Cloud,NFVI,Solution,7,NFVI,NFVI,Use the PodSpec to set limits to help minimize...,Ericsson uses CPU pinning and resource dedicat...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav


In [14]:
soc_small["requirement"].head(20)

0        Restrict containers running in privileged mode
1            Do not run processes in containers as root
2                          Restrict the use of hostpath
3     Where possible, Cryptographically isolate crit...
4     Use the PodSpec to set limits to help minimize...
5     Setting a resource quota or creating a limit r...
6     Harden and optimize operating system for runni...
7     Implement an immutable infrastructure and auto...
8     Maintain a golden (configured and clean) set o...
9      Harden Kubernetes clusters using CIS guidelines 
10    Minimize direct access to worker nodes risk by...
11    Supplier should provide best practices to ensu...
12    Container encryption/ decryption of images on ...
13    Build images from scratch or create minimal de...
14    Scan container images for vulnerabilities regu...
15      Restrict access to container image repositories
16    Container Images build should ensure that the ...
17                    Container images should be

In [19]:
soc.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4155 entries, 0 to 4154
Columns: 17 entries, Domain to Responded By
dtypes: int64(1), object(16)
memory usage: 6.1 MB


In [9]:
#using soc csv file dataset (trial with top 5 questions)
#updated with gradio

from transformers import AutoTokenizer, AutoModel
import torch
import gradio as gr
import torch.nn.functional as F
import pandas as pd
import numpy as np
#from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity



#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)



# Convert the questions in the training dataset to sentence embeddings
questions = soc_small['requirement'].tolist()
answers = soc_small['response'].tolist()



# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
# Load the pre-trained Sentence Transformer
#model = SentenceTransformer('all-MiniLM-L6-v2')

model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(questions, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# Function to find the most similar question in the training dataset to a user input question
def find_similar_question(input_question):
    encoded_input_question = tokenizer(input_question, padding=True, truncation=True, return_tensors='pt')
    # Compute token embeddings
    with torch.no_grad():
      model_output_input_q = model(**encoded_input_question)

    # Perform pooling
    sentence_embeddings_input = mean_pooling(model_output_input_q, encoded_input_question['attention_mask'])

    cosine_similarities = cosine_similarity(sentence_embeddings_input, sentence_embeddings).flatten()
    most_similar_index = cosine_similarities.argmax()
    most_similar_question = questions[most_similar_index]
    most_similar_answer = answers[most_similar_index]
    most_similar_compliancy = answers[most_similar_index]
    score = cosine_similarities[most_similar_index]
    top_5_indices = np.argsort(cosine_similarities)[::-1][:5]
    top_5_questions = [questions[i] for i in top_5_indices]
    top_5_scores = [cosine_similarities[i] for i in top_5_indices]
    return most_similar_question, most_similar_answer, score, top_5_questions, top_5_scores





In [10]:
# Define the gradio interface
def chatbot(input_question):
    most_similar_question, most_similar_answer, score, top_5_questions, top_5_scores = find_similar_question(input_question)
    return most_similar_question, most_similar_answer, score, top_5_questions, top_5_scores


# Launch the demo version of the chatbot using gradio
gr.Interface(fn=chatbot, inputs=gr.inputs.Textbox(label="Ask a question"),outputs=[gr.outputs.Textbox(label="most_similar_question"),gr.outputs.Textbox(label="most similar answer"),gr.outputs.Textbox(label="Relevence score"),gr.outputs.Textbox(label="top 5 related questions"),gr.outputs.Textbox(label="relevence score of top 5")] ,title=" SoC App").launch()


C:\Users\egernit\Anaconda3\envs\myenv\lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\egernit\Anaconda3\envs\myenv\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\egernit\Anaconda3\envs\myenv\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\egernit\Anaconda3\envs\myenv\lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [76]:
soc_small.columns.to_list()

['domain',
 'solution_area',
 'product',
 'produt_release',
 'solution_category',
 'deployment_model',
 'requirement',
 'response',
 'compliance',
 'full_bid_name',
 'bid_id_only',
 'day_marked_completed',
 'customer',
 'country',
 'market_area',
 'bid_type',
 'responded_by']

In [15]:
#using soc csv file dataset
#updated with gradio
# Displaying only answer and requirements
#use soc small dataset(1000 q)

from transformers import AutoTokenizer, AutoModel
import torch
import gradio as gr
import torch.nn.functional as F
import pandas as pd
import numpy as np
#from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity



#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)



# Convert the questions in the training dataset to sentence embeddings
questions = soc_small['requirement'].tolist()
answers = soc_small['response'].tolist()
compliance = soc_small['compliance'].tolist()
product = soc_small['deployment_model'].tolist()
product_release = soc_small['produt_release'].tolist()
time = soc_small['day_marked_completed'].tolist()
bid_details = soc_small['full_bid_name'].tolist()

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
# Load the pre-trained Sentence Transformer
#model = SentenceTransformer('all-MiniLM-L6-v2')

model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(questions, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# Function to find the most similar question in the training dataset to a user input question
def find_similar_question(input_question):
    encoded_input_question = tokenizer(input_question, padding=True, truncation=True, return_tensors='pt')
    # Compute token embeddings
    with torch.no_grad():
      model_output_input_q = model(**encoded_input_question)

    # Perform pooling
    sentence_embeddings_input = mean_pooling(model_output_input_q, encoded_input_question['attention_mask'])

    cosine_similarities = cosine_similarity(sentence_embeddings_input, sentence_embeddings).flatten()
    most_similar_index = cosine_similarities.argmax()
    most_similar_question = questions[most_similar_index]
    most_similar_answer = answers[most_similar_index]
    score = round(cosine_similarities[most_similar_index],2)
    comp=compliance[most_similar_index]
    prod=product[most_similar_index]
    prod_rel=product_release[most_similar_index]
    bid=bid_details[most_similar_index]
    answer_time=time[most_similar_index]
    top_5_indices = np.argsort(cosine_similarities)[::-1][:5]
    top_5_questions = [questions[i] for i in top_5_indices]
    top_5_scores = [cosine_similarities[i] for i in top_5_indices]
    top_5_answers = [answers[i] for i in top_5_indices]
    relevent_answer_1=top_5_answers[2]
    relevent_answer_2=top_5_answers[3]
    relevent_answer_3=top_5_answers[4]
    return most_similar_question, most_similar_answer, score, comp,prod,prod_rel,bid,answer_time,relevent_answer_1,relevent_answer_2,relevent_answer_3
            



In [1]:
# Define the gradio interface for our use case
def chat_bot(input_question):
    most_similar_question, most_similar_answer, score, comp,prod,prod_rel,bid,answer_time,relevent_answer_1,relevent_answer_2,relevent_answer_3 = find_similar_question(input_question)
    return most_similar_question, most_similar_answer, score, comp,prod,prod_rel,bid,answer_time,relevent_answer_1,relevent_answer_2,relevent_answer_3



# Launch the demo version of the chatbot using gradio
gr.Interface(fn=chat_bot, inputs=gr.inputs.Textbox(label="Ask a question"),outputs=[gr.outputs.Textbox(label="most relevent question"),gr.outputs.Textbox(label="most relevent answer"),
                                                    gr.outputs.Textbox(label="Relevence score"),gr.outputs.Textbox(label="compliance provided"),gr.outputs.Textbox(label="Product"),gr.outputs.Textbox(label="Product Release"),gr.outputs.Textbox(label="bid details"),gr.outputs.Textbox(label="answering timeline"),gr.outputs.Textbox(label="other relevent answer"),gr.outputs.Textbox(label="other relevent answer"),gr.outputs.Textbox(label="other relevent answer")] ,title=" SoC App").launch()



NameError: name 'gr' is not defined

In [59]:
soc_clean.columns.to_list()

['domain',
 'solution_area',
 'product',
 'produt_release',
 'solution_category',
 'deployment_model',
 'requirement',
 'response',
 'compliance',
 'full_bid_name',
 'bid_id_only',
 'day_marked_completed',
 'customer',
 'country',
 'market_area',
 'bid_type',
 'responded_by']

In [68]:
soc_clean.shape
soc_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4155 entries, 0 to 4154
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   domain                4155 non-null   object
 1   solution_area         4155 non-null   object
 2   product               4155 non-null   object
 3   produt_release        4155 non-null   object
 4   solution_category     4155 non-null   object
 5   deployment_model      4155 non-null   object
 6   requirement           4155 non-null   object
 7   response              4155 non-null   object
 8   compliance            4155 non-null   object
 9   full_bid_name         4155 non-null   object
 10  bid_id_only           4155 non-null   object
 11  day_marked_completed  4155 non-null   object
 12  customer              4155 non-null   object
 13  country               4155 non-null   object
 14  market_area           4155 non-null   object
 15  bid_type              4155 non-null   

In [62]:
soc_clean.head()

,domain,solution_area,product,produt_release,solution_category,deployment_model,requirement,response,compliance,full_bid_name,bid_id_only,day_marked_completed,customer,country,market_area,bid_type,responded_by
0,Cloud,NFVI,Solution,7,NFVI,NFVI,Restrict containers running in privileged mode,Containers can be restricted to be running in ...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
1,Cloud,NFVI,Solution,7,NFVI,NFVI,Do not run processes in containers as root,Presently docker deamon is running as root but...,NC,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
2,Cloud,NFVI,Solution,7,NFVI,NFVI,Restrict the use of hostpath,It is possible for a container to breakout of ...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
3,Cloud,NFVI,Solution,7,NFVI,NFVI,"Where possible, Cryptographically isolate crit...",Where possible Ericsson will try to implement ...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav
4,Cloud,NFVI,Solution,7,NFVI,NFVI,Use the PodSpec to set limits to help minimize...,Ericsson uses CPU pinning and resource dedicat...,C,RFP_MMEA_JORDAN_MobileCOM_34125,34125,Q2-2022,Mobilecom,Jordan,MMEA,RFP,ekumgav


In [72]:
#using soc csv file dataset
#updated with gradio
# with complete soc for 2021 and 2022

from transformers import AutoTokenizer, AutoModel
import torch
import gradio as gr
import torch.nn.functional as F
import pandas as pd
import numpy as np
#from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity



#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)



# Convert the questions in the training dataset to sentence embeddings
questions = soc_clean['requirement'].tolist()
answers = soc_clean['response'].tolist()
compliance = soc_clean['compliance'].tolist()
product = soc_clean['deployment_model'].tolist()
product_release = soc_clean['produt_release'].tolist()
time = soc_clean['day_marked_completed'].tolist()
bid_details = soc_clean['full_bid_name'].tolist()

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
# Load the pre-trained Sentence Transformer
#model = SentenceTransformer('all-MiniLM-L6-v2')

model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(questions, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# Function to find the most similar question in the training dataset to a user input question
def find_similar_question(input_question):
    encoded_input_question = tokenizer(input_question, padding=True, truncation=True, return_tensors='pt')
    # Compute token embeddings
    with torch.no_grad():
      model_output_input_q = model(**encoded_input_question)

    # Perform pooling
    sentence_embeddings_input = mean_pooling(model_output_input_q, encoded_input_question['attention_mask'])

    cosine_similarities = cosine_similarity(sentence_embeddings_input, sentence_embeddings).flatten()
    most_similar_index = cosine_similarities.argmax()
    most_similar_question = questions[most_similar_index]
    most_similar_answer = answers[most_similar_index]
    score = round(cosine_similarities[most_similar_index],2)
    comp=compliance[most_similar_index]
    prod=product[most_similar_index]
    prod_rel=product_release[most_similar_index]
    bid=bid_details[most_similar_index]
    answer_time=time[most_similar_index]
    top_5_indices = np.argsort(cosine_similarities)[::-1][:5]
    top_5_questions = [questions[i] for i in top_5_indices]
    top_5_scores = [cosine_similarities[i] for i in top_5_indices]
    top_5_answers = [answers[i] for i in top_5_indices]
    relevent_answer_1=top_5_answers[2]
    relevent_answer_2=top_5_answers[3]
    relevent_answer_3=top_5_answers[4]
    return most_similar_question, most_similar_answer, score, comp,prod,prod_rel,bid,answer_time,relevent_answer_1,relevent_answer_2,relevent_answer_3
            



RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 52281999360 bytes.